# Exponential Smoothing and Holt-Winters Forecasting

Exponential smoothing models forecast by combining level, trend, and seasonality with exponentially decaying weights. This notebook builds intuition, shows the core equations, and fits models with `statsmodels` plus Plotly visuals.


## Takeaways
- Exponential smoothing is a fast, interpretable baseline for trend and seasonal data.
- Holt's method extends SES with trend; Holt-Winters adds seasonality.
- Additive seasonality fits constant seasonal amplitude; multiplicative fits proportional amplitude.
- Damped trends often improve long-horizon forecasts.


## Core idea
Simple exponential smoothing (SES) keeps a single **level** state and discounts older observations:

\[
\ell_t = \alpha y_t + (1-\alpha)\ell_{t-1}
\]
\[
\hat{y}_{t+h} = \ell_t
\]

Here, \(\alpha \in (0,1)\) controls how quickly the model reacts to new data.


### Holt's linear trend
Adds a **trend** state \(b_t\):

\[
\ell_t = \alpha y_t + (1-\alpha)(\ell_{t-1}+b_{t-1})
\]
\[
b_t = \beta (\ell_t-\ell_{t-1}) + (1-\beta)b_{t-1}
\]
\[
\hat{y}_{t+h} = \ell_t + h b_t
\]

Damped trend introduces a factor \(\phi \in (0,1)\) so the trend contribution decays for long horizons.


### Holt-Winters seasonality
For **additive** seasonality with period \(m\):

\[
\ell_t = \alpha (y_t - s_{t-m}) + (1-\alpha)(\ell_{t-1}+b_{t-1})
\]
\[
b_t = \beta (\ell_t-\ell_{t-1}) + (1-\beta)b_{t-1}
\]
\[
s_t = \gamma (y_t-\ell_t) + (1-\gamma)s_{t-m}
\]
\[
\hat{y}_{t+h} = \ell_t + h b_t + s_{t-m+h}
\]

For **multiplicative** seasonality, replace \(y_t - s_{t-m}\) with \(y_t / s_{t-m}\) and \(+ s_{t-m+h}\) with \(\times s_{t-m+h}\).


## When to use
- Short to medium horizons.
- Clear trend and/or seasonality.
- You want fast, interpretable baselines before heavier models.


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import ExponentialSmoothing


## Forecasting workflow

A lightweight workflow for exponential smoothing projects.


In [ ]:
steps = ["Series", "Pick components", "Fit smoothing", "Forecast", "Evaluate"]

fig = go.Figure()
for i, step in enumerate(steps):
    x0 = i * 1.6
    x1 = x0 + 1.3
    fig.add_shape(
        type="rect",
        x0=x0,
        x1=x1,
        y0=0,
        y1=0.6,
        line=dict(color="#2f2f2f"),
        fillcolor="rgba(80, 80, 80, 0.08)",
    )
    fig.add_annotation(x=(x0 + x1) / 2, y=0.3, text=step, showarrow=False)
    if i < len(steps) - 1:
        fig.add_annotation(x=x1 + 0.15, y=0.3, text="->", showarrow=False, font=dict(size=18))

fig.update_layout(
    title="Exponential smoothing workflow",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    height=260,
    margin=dict(l=20, r=20, t=60, b=20),
    template="plotly_white",
)
fig


## A synthetic seasonal series


In [ ]:
# Reproducible seasonal series with trend
rng = np.random.default_rng(7)
period = 12
n_obs = 72

t = np.arange(n_obs)
trend = 0.4 * t
season = 8 * np.sin(2 * np.pi * t / period) + 2 * np.cos(2 * np.pi * t / period)
noise = rng.normal(0, 2.0, n_obs)

series = pd.Series(
    50 + trend + season + noise,
    index=pd.date_range("2018-01-01", periods=n_obs, freq="MS"),
    name="sales",
)
series.head()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index, y=series, mode="lines", name="series"))
fig.update_layout(
    title="Synthetic monthly series",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white",
)
fig


## Exponential weights intuition


In [ ]:
def exp_weights(alpha: float, n_lags: int = 24) -> pd.DataFrame:
    lags = np.arange(n_lags)
    weights = alpha * (1 - alpha) ** lags
    return pd.DataFrame({"lag": lags, "weight": weights})

fig = go.Figure()
for alpha in [0.2, 0.5, 0.8]:
    w = exp_weights(alpha)
    fig.add_trace(
        go.Scatter(
            x=w["lag"],
            y=w["weight"],
            mode="lines+markers",
            name=f"alpha={alpha}",
        )
    )

fig.update_layout(
    title="Exponential smoothing weights",
    xaxis_title="Lag",
    yaxis_title="Weight",
    template="plotly_white",
)
fig


## Simple exponential smoothing from scratch


In [ ]:
def simple_exponential_smoothing(series: pd.Series, alpha: float) -> pd.Series:
    # Return fitted level values for simple exponential smoothing.
    level = series.iloc[0]
    fitted = []
    for value in series:
        level = alpha * value + (1 - alpha) * level
        fitted.append(level)
    return pd.Series(fitted, index=series.index, name=f"SES(alpha={alpha})")

ses_slow = simple_exponential_smoothing(series, alpha=0.2)
ses_fast = simple_exponential_smoothing(series, alpha=0.8)

fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index, y=series, mode="lines", name="series"))
fig.add_trace(go.Scatter(x=ses_slow.index, y=ses_slow, mode="lines", name="SES alpha=0.2"))
fig.add_trace(go.Scatter(x=ses_fast.index, y=ses_fast, mode="lines", name="SES alpha=0.8"))
fig.update_layout(
    title="SES smoothing strength",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white",
)
fig


## Holt-Winters in statsmodels


In [ ]:
horizon = 12
train = series.iloc[:-horizon]
test = series.iloc[-horizon:]

ses_model = ExponentialSmoothing(
    train,
    trend=None,
    seasonal=None,
    initialization_method="estimated",
).fit(optimized=True)

holt_model = ExponentialSmoothing(
    train,
    trend="add",
    damped_trend=True,
    seasonal=None,
    initialization_method="estimated",
).fit(optimized=True)

hw_model = ExponentialSmoothing(
    train,
    trend="add",
    damped_trend=True,
    seasonal="add",
    seasonal_periods=period,
    initialization_method="estimated",
).fit(optimized=True)

ses_forecast = ses_model.forecast(horizon)
holt_forecast = holt_model.forecast(horizon)
hw_forecast = hw_model.forecast(horizon)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode="lines", name="train"))
fig.add_trace(go.Scatter(x=test.index, y=test, mode="lines", name="test"))

fig.add_trace(go.Scatter(x=test.index, y=ses_forecast, mode="lines", name="SES forecast"))
fig.add_trace(go.Scatter(x=test.index, y=holt_forecast, mode="lines", name="Holt forecast"))
fig.add_trace(go.Scatter(x=test.index, y=hw_forecast, mode="lines", name="Holt-Winters forecast"))

fig.add_vline(x=test.index[0], line_dash="dash", line_color="gray")
fig.update_layout(
    title="Forecast comparison",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white",
)
fig


## A simple baseline and error metrics


In [ ]:
def seasonal_naive(train: pd.Series, horizon: int, period: int) -> pd.Series:
    last_season = train.iloc[-period:]
    reps = int(np.ceil(horizon / period))
    values = np.tile(last_season.values, reps)[:horizon]
    return pd.Series(values, index=test.index, name="seasonal_naive")


def mae(y_true, y_pred) -> float:
    return float(np.mean(np.abs(np.asarray(y_true) - np.asarray(y_pred))))


def mape(y_true, y_pred) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return float("nan")
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100)

baseline = seasonal_naive(train, horizon, period)

results = pd.DataFrame(
    {
        "MAE": [
            mae(test, baseline),
            mae(test, ses_forecast),
            mae(test, holt_forecast),
            mae(test, hw_forecast),
        ],
        "MAPE (%)": [
            mape(test, baseline),
            mape(test, ses_forecast),
            mape(test, holt_forecast),
            mape(test, hw_forecast),
        ],
    },
    index=["Seasonal naive", "SES", "Holt", "Holt-Winters"],
)
results


## Practical tips
- Use additive seasonality for roughly constant seasonal amplitude; use multiplicative (or a log transform) when the amplitude scales with the level.
- Always sanity-check the season length. Monthly data usually implies `seasonal_periods=12`, weekly data might be 52.
- Damped trends often reduce overly optimistic long-horizon forecasts.
- Compare against naive or seasonal naive baselines; exponential smoothing should beat them consistently.


## Exercises
1. Swap to multiplicative seasonality and compare errors.
2. Try a longer horizon (24 months). Does damping help?
3. Replace the synthetic series with a real dataset and tune the model.


## Further reading
- `statsmodels` Holt-Winters documentation
- Hyndman, R.J. and Athanasopoulos, G. *Forecasting: Principles and Practice* (ETS chapter)
